## DETECTING SPAM MAILS USING KERAS AND GLOVE6B WORD EMBEDDINGS

In [45]:
import tqdm
import numpy as np
import keras_metrics # for recall and precision metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dropout, Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pickle
print('Modules imported')

Modules imported


In [2]:
# Parameters
SEQUENCE_LENGTH = 100 # the length of all sequences (number of words per sample)
EMBEDDING_SIZE = 100  # Using 100-Dimensional GloVe embedding vectors
TEST_SIZE = 0.25 # ratio of testing set

BATCH_SIZE = 64
EPOCHS = 20 # number of epochs

# to convert labels to integers and vice-versa
label2int = {"ham": 0, "spam": 1}
int2label = {0: "ham", 1: "spam"}

#  Load dataset

In [3]:
def load_data():
    """
    Loads SMS Spam Collection dataset
    """
    texts, labels = [], []
    with open("data/SMSSpamCollection") as f:
        for line in f:
            split = line.split()
            labels.append(split[0].strip())
            texts.append(' '.join(split[1:]).strip())
    return texts, labels

In [4]:
X, y = load_data()
print(X[:3])
print(y[:3])

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
['ham', 'ham', 'spam']


# Preparing the Dataset

Now, we need a way to vectorize the text corpus by turning each text into a sequence of integers, you're now may be wondering why we need to turn the text into sequence of integers, well, remember we are going to feed the text into a neural network, a neural network only understands numbers. More precisely, a fixed length sequence of integers.

But before we do all of that, we need to clean this corpus by removing punctuations, lowercase all characters, etc. Luckily for us, Keras has a builtin class keras.preprocessing.text.Tokenizer() that does all that in few lines of code:

In [5]:
# Text tokenization
# vectorizing text, turning each text into sequence of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
# convert to sequence of integers
X = tokenizer.texts_to_sequences(X)

In [9]:
X[0]

[49,
 472,
 4436,
 843,
 756,
 659,
 64,
 8,
 1328,
 87,
 123,
 352,
 1329,
 148,
 2996,
 1330,
 67,
 58,
 4437,
 144]

In [10]:
# convert to numpy arrays
X = np.array(X)
y = np.array(y)
# pad sequences at the beginning of each sequence with 0's
# for example if SEQUENCE_LENGTH=4:
# [[5, 3, 2], [5, 1, 2, 3], [3, 4]]
# will be transformed to:
# [[0, 5, 3, 2], [5, 1, 2, 3], [0, 0, 3, 4]]
X = pad_sequences(X, maxlen=SEQUENCE_LENGTH)

In [11]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,   49,  472, 4436,  843,  756,  659,   64,    8,
       1328,   87,  123,  352, 1329,  148, 2996, 1330,   67,   58, 4437,
        144])

As you may remember, we set SEQUENCE_LENGTH to 100, in this way, all sequences have a length of 100.

Now our labels are text also, but we gonna make a different approach here, since the labels are only "spam" and "ham", we need to one-hot encode them:

In [12]:
# One Hot encoding labels
# [spam, ham, spam, ham, ham] will be converted to:
# [1, 0, 1, 0, 1] and then to:
# [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]

y = [ label2int[label] for label in y ]
y = to_categorical(y)

In [13]:
y[0]

array([1., 0.], dtype=float32)

In [14]:
# Spiliting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=7)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4180, 100)
(1394, 100)
(4180, 2)
(1394, 2)


# Building the Model
<img src='data/img/network_diagram.png' height=500 width=500/> ![LSTM nueral net](data/img/network_diagram.png)

The first layer is a pre-trained embedding layer that maps each word to a N-dimensional vector of real numbers ( the EMBEDDING_SIZE corresponds to the size of this vector, in this case 100). Two words that have similar meaning tend to have very close vectors.

In [17]:
def get_embedding_vectors(tokenizer, dim=100):
    embedding_index = {}
    with open(f"data/glove.6B.{dim}d.txt", encoding='utf8') as f:
        for line in tqdm.tqdm(f, "Reading GloVe"):
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = vectors

    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index)+1, dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # words not found will be 0s
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In order to download the glove.6b pre-trained model follow the below link
http://nlp.stanford.edu/data/glove.6B.zip

# Define a model

In [19]:
def get_model(tokenizer, lstm_units):
    """
    Constructs the model,
    Embedding vectors => LSTM => 2 output Fully-Connected neurons with softmax activation
    """
    # get the GloVe embedding vectors
    embedding_matrix = get_embedding_vectors(tokenizer)
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index)+1,
              EMBEDDING_SIZE,
              weights=[embedding_matrix],
              trainable=False,
              input_length=SEQUENCE_LENGTH))

    model.add(LSTM(lstm_units, recurrent_dropout=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation="softmax"))
    # compile as rmsprop optimizer
    # aswell as with recall metric
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",
                  metrics=["accuracy", keras_metrics.precision(), keras_metrics.recall()])
    model.summary()
    return model

The above function constructs the whole model, we loaded the pre-trained embedding vectors to the Embedding layer, and set trainable=False, this will freeze the embedding weights during the training process.


<img src='data/img/definition.png' height=500 width=500/> ![LSTM nueral net](data/img/definition.png)

In [20]:
model = get_model(tokenizer=tokenizer, lstm_units=128)

Reading GloVe: 400000it [00:34, 11491.43it/s]


tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          901300    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 1,018,806
Trainable params: 117,506
Non-trainable params: 901,300
____________________________

In [22]:
# initialize our ModelCheckpoint and TensorBoard callbacks
# model checkpoint for saving best weights
model_checkpoint = ModelCheckpoint("results/spam_classifier_{val_loss:.2f}", save_best_only=True,
                                    verbose=1)
# for better visualization
tensorboard = TensorBoard(f"logs/spam_classifier_{time.time()}")
# print our data shapes
print("X_train.shape:", X_train.shape)
print("X_test.shape:", X_test.shape)
print("y_train.shape:", y_train.shape)
print("y_test.shape:", y_test.shape)
# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          batch_size=BATCH_SIZE, epochs=EPOCHS,
          callbacks=[tensorboard, model_checkpoint],
          verbose=1)

X_train.shape: (4180, 100)
X_test.shape: (1394, 100)
y_train.shape: (4180, 2)
y_test.shape: (1394, 2)
Train on 4180 samples, validate on 1394 samples
Epoch 1/20
 128/4180 [..............................] - ETA: 1:08 - loss: 0.0881 - accuracy: 0.9531 - precision: 0.9492 - recall: 1.000 - ETA: 53s - loss: 0.0800 - accuracy: 0.9688 - precision: 0.9615 - recall: 0.9956 

D:\Intalled Softwares\Anaconda\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.931077). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


4180/4180 [==============================] - ETA: 1:22 - loss: 0.1046 - accuracy: 0.9688 - precision: 0.9649 - recall: 0.995 - ETA: 1:08 - loss: 0.0923 - accuracy: 0.9727 - precision: 0.9683 - recall: 0.994 - ETA: 59s - loss: 0.1236 - accuracy: 0.9625 - precision: 0.9679 - recall: 0.993 - ETA: 53s - loss: 0.1191 - accuracy: 0.9635 - precision: 0.9684 - recall: 0.99 - ETA: 49s - loss: 0.1142 - accuracy: 0.9643 - precision: 0.9690 - recall: 0.99 - ETA: 45s - loss: 0.1070 - accuracy: 0.9668 - precision: 0.9696 - recall: 0.99 - ETA: 42s - loss: 0.0988 - accuracy: 0.9705 - precision: 0.9704 - recall: 0.99 - ETA: 40s - loss: 0.1028 - accuracy: 0.9688 - precision: 0.9708 - recall: 0.99 - ETA: 38s - loss: 0.1025 - accuracy: 0.9688 - precision: 0.9713 - recall: 0.99 - ETA: 36s - loss: 0.1035 - accuracy: 0.9688 - precision: 0.9716 - recall: 0.99 - ETA: 35s - loss: 0.1040 - accuracy: 0.9688 - precision: 0.9719 - recall: 0.99 - ETA: 33s - loss: 0.1025 - accuracy: 0.9699 - precision: 0.9722 - recal

4180/4180 [==============================] - ETA: 40s - loss: 0.0508 - accuracy: 0.9844 - precision: 1.0000 - recall: 0.98 - ETA: 39s - loss: 0.0615 - accuracy: 0.9688 - precision: 0.9868 - recall: 0.98 - ETA: 40s - loss: 0.0763 - accuracy: 0.9740 - precision: 0.9835 - recall: 0.98 - ETA: 42s - loss: 0.0868 - accuracy: 0.9688 - precision: 0.9821 - recall: 0.98 - ETA: 44s - loss: 0.0890 - accuracy: 0.9656 - precision: 0.9801 - recall: 0.98 - ETA: 46s - loss: 0.0848 - accuracy: 0.9714 - precision: 0.9795 - recall: 0.98 - ETA: 46s - loss: 0.0793 - accuracy: 0.9732 - precision: 0.9796 - recall: 0.98 - ETA: 44s - loss: 0.0859 - accuracy: 0.9727 - precision: 0.9794 - recall: 0.98 - ETA: 42s - loss: 0.0833 - accuracy: 0.9740 - precision: 0.9793 - recall: 0.98 - ETA: 42s - loss: 0.0798 - accuracy: 0.9750 - precision: 0.9793 - recall: 0.99 - ETA: 42s - loss: 0.0805 - accuracy: 0.9759 - precision: 0.9793 - recall: 0.99 - ETA: 42s - loss: 0.0839 - accuracy: 0.9740 - precision: 0.9793 - recall: 0.

4180/4180 [==============================] - ETA: 44s - loss: 0.0427 - accuracy: 0.9844 - precision: 1.0000 - recall: 0.98 - ETA: 43s - loss: 0.0761 - accuracy: 0.9688 - precision: 0.9953 - recall: 0.97 - ETA: 49s - loss: 0.0645 - accuracy: 0.9740 - precision: 0.9927 - recall: 0.97 - ETA: 53s - loss: 0.0628 - accuracy: 0.9766 - precision: 0.9911 - recall: 0.98 - ETA: 53s - loss: 0.0691 - accuracy: 0.9750 - precision: 0.9892 - recall: 0.98 - ETA: 53s - loss: 0.0726 - accuracy: 0.9766 - precision: 0.9885 - recall: 0.98 - ETA: 53s - loss: 0.0649 - accuracy: 0.9799 - precision: 0.9883 - recall: 0.98 - ETA: 52s - loss: 0.0589 - accuracy: 0.9824 - precision: 0.9883 - recall: 0.98 - ETA: 52s - loss: 0.0684 - accuracy: 0.9809 - precision: 0.9883 - recall: 0.98 - ETA: 51s - loss: 0.0668 - accuracy: 0.9812 - precision: 0.9882 - recall: 0.98 - ETA: 49s - loss: 0.0648 - accuracy: 0.9815 - precision: 0.9882 - recall: 0.98 - ETA: 48s - loss: 0.0608 - accuracy: 0.9831 - precision: 0.9883 - recall: 0.

...

4180/4180 [==============================] - ETA: 37s - loss: 0.0033 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 32s - loss: 0.0251 - accuracy: 0.9922 - precision: 0.9952 - recall: 1.00 - ETA: 31s - loss: 0.0198 - accuracy: 0.9948 - precision: 0.9947 - recall: 1.00 - ETA: 31s - loss: 0.0169 - accuracy: 0.9961 - precision: 0.9949 - recall: 1.00 - ETA: 31s - loss: 0.0197 - accuracy: 0.9937 - precision: 0.9944 - recall: 1.00 - ETA: 31s - loss: 0.0172 - accuracy: 0.9948 - precision: 0.9943 - recall: 1.00 - ETA: 30s - loss: 0.0178 - accuracy: 0.9955 - precision: 0.9944 - recall: 1.00 - ETA: 29s - loss: 0.0163 - accuracy: 0.9961 - precision: 0.9945 - recall: 1.00 - ETA: 28s - loss: 0.0151 - accuracy: 0.9965 - precision: 0.9946 - recall: 1.00 - ETA: 27s - loss: 0.0156 - accuracy: 0.9953 - precision: 0.9946 - recall: 1.00 - ETA: 27s - loss: 0.0170 - accuracy: 0.9929 - precision: 0.9947 - recall: 0.99 - ETA: 26s - loss: 0.0163 - accuracy: 0.9935 - precision: 0.9947 - recall: 0.

4180/4180 [==============================] - ETA: 46s - loss: 0.0266 - accuracy: 0.9844 - precision: 0.9808 - recall: 1.00 - ETA: 36s - loss: 0.0176 - accuracy: 0.9922 - precision: 0.9856 - recall: 1.00 - ETA: 34s - loss: 0.0138 - accuracy: 0.9948 - precision: 0.9883 - recall: 1.00 - ETA: 35s - loss: 0.0107 - accuracy: 0.9961 - precision: 0.9900 - recall: 1.00 - ETA: 34s - loss: 0.0100 - accuracy: 0.9969 - precision: 0.9913 - recall: 1.00 - ETA: 33s - loss: 0.0094 - accuracy: 0.9974 - precision: 0.9922 - recall: 1.00 - ETA: 33s - loss: 0.0083 - accuracy: 0.9978 - precision: 0.9930 - recall: 1.00 - ETA: 33s - loss: 0.0082 - accuracy: 0.9980 - precision: 0.9936 - recall: 1.00 - ETA: 33s - loss: 0.0099 - accuracy: 0.9965 - precision: 0.9938 - recall: 1.00 - ETA: 31s - loss: 0.0090 - accuracy: 0.9969 - precision: 0.9941 - recall: 1.00 - ETA: 30s - loss: 0.0083 - accuracy: 0.9972 - precision: 0.9943 - recall: 1.00 - ETA: 29s - loss: 0.0076 - accuracy: 0.9974 - precision: 0.9946 - recall: 1.

4180/4180 [==============================] - ETA: 1:08 - loss: 0.0024 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.000 - ETA: 1:06 - loss: 0.0045 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.000 - ETA: 1:02 - loss: 0.0052 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.000 - ETA: 53s - loss: 0.0051 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.000 - ETA: 48s - loss: 0.0044 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 44s - loss: 0.0038 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 41s - loss: 0.0043 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 40s - loss: 0.0039 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 38s - loss: 0.0048 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 36s - loss: 0.0043 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 34s - loss: 0.0039 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 33s - loss: 0.0037 - accuracy: 1.0000 - precision: 1.0000 - rec

4180/4180 [==============================] - ETA: 58s - loss: 0.0036 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 45s - loss: 0.0034 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 40s - loss: 0.0029 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 38s - loss: 0.0084 - accuracy: 0.9961 - precision: 0.9989 - recall: 1.00 - ETA: 35s - loss: 0.0080 - accuracy: 0.9969 - precision: 0.9984 - recall: 1.00 - ETA: 34s - loss: 0.0088 - accuracy: 0.9974 - precision: 0.9982 - recall: 1.00 - ETA: 33s - loss: 0.0079 - accuracy: 0.9978 - precision: 0.9981 - recall: 1.00 - ETA: 34s - loss: 0.0079 - accuracy: 0.9980 - precision: 0.9981 - recall: 1.00 - ETA: 33s - loss: 0.0073 - accuracy: 0.9983 - precision: 0.9981 - recall: 1.00 - ETA: 32s - loss: 0.0066 - accuracy: 0.9984 - precision: 0.9981 - recall: 1.00 - ETA: 32s - loss: 0.0060 - accuracy: 0.9986 - precision: 0.9981 - recall: 1.00 - ETA: 31s - loss: 0.0061 - accuracy: 0.9987 - precision: 0.9981 - recall: 1.

4180/4180 [==============================] - ETA: 31s - loss: 0.0019 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 30s - loss: 0.0013 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.00 - ETA: 29s - loss: 0.0063 - accuracy: 0.9948 - precision: 0.9981 - recall: 1.00 - ETA: 29s - loss: 0.0087 - accuracy: 0.9922 - precision: 0.9964 - recall: 1.00 - ETA: 28s - loss: 0.0076 - accuracy: 0.9937 - precision: 0.9957 - recall: 1.00 - ETA: 28s - loss: 0.0088 - accuracy: 0.9948 - precision: 0.9955 - recall: 1.00 - ETA: 27s - loss: 0.0076 - accuracy: 0.9955 - precision: 0.9954 - recall: 1.00 - ETA: 27s - loss: 0.0068 - accuracy: 0.9961 - precision: 0.9954 - recall: 1.00 - ETA: 27s - loss: 0.0062 - accuracy: 0.9965 - precision: 0.9955 - recall: 1.00 - ETA: 27s - loss: 0.0056 - accuracy: 0.9969 - precision: 0.9956 - recall: 1.00 - ETA: 27s - loss: 0.0053 - accuracy: 0.9972 - precision: 0.9957 - recall: 1.00 - ETA: 27s - loss: 0.0049 - accuracy: 0.9974 - precision: 0.9958 - recall: 1.

In [37]:
model.save('spam_classifier_20_epoch.h5')

In [31]:
# get the loss and metrics
result = model.evaluate(X_test, y_test)
# extract those
loss = result[0]
accuracy = result[1]
precision = result[2]
recall = result[3]

print(f"[+] Accuracy: {accuracy*100:.2f}%")
print(f"[+] Precision:   {precision*100:.2f}%")
print(f"[+] Recall:   {recall*100:.2f}%")

1394/1394 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 1ms/step
[+] Accuracy: 98.49%
[+] Precision:   98.94%
[+] Recall:   99.60%


In [50]:
def get_predictions(text):
    model.load_weights("spam_classifier_20_epoch.h5")
    sequence = tokenizer.texts_to_sequences([text])
    # pad the sequence
    sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
    # get the prediction
    prediction = model.predict(sequence)[0]
    # one-hot encoded vector, revert using np.argmax
    return int2label[np.argmax(prediction)]

In [51]:
text = "Congratulations! you have won 100,000$ this week, click here to claim fast"
print(get_predictions(text))

text = "Hi man, Meet you tommorow 3.00 pm"
print(get_predictions(text))

spam
ham
